## SimSiamの写経  
https://www.guruguru.science/competitions/17/discussions/a39d588e-aff2-4728-8323-b07f15563552/

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install lightly

     |████████████████████████████████| 240 kB 9.2 MB/s 
     |████████████████████████████████| 76 kB 3.4 MB/s 
     |████████████████████████████████| 813 kB 14.6 MB/s 
     |████████████████████████████████| 144 kB 20.4 MB/s 
     |████████████████████████████████| 112 kB 36.6 MB/s 
     |████████████████████████████████| 74 kB 2.9 MB/s 
     |████████████████████████████████| 636 kB 36.2 MB/s 
     |████████████████████████████████| 10.6 MB 38.9 MB/s 
     |████████████████████████████████| 118 kB 52.4 MB/s 
     |████████████████████████████████| 234 kB 50.9 MB/s 
     |████████████████████████████████| 829 kB 38.6 MB/s 
     |████████████████████████████████| 1.3 MB 32.6 MB/s 
     |████████████████████████████████| 294 kB 47.6 MB/s 
     |████████████████████████████████| 142 kB 57.3 MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141229 sha256=9c801a613caf97bd2a0025eea2ba8a0978585a9b37428af9cec48f756563801f
  Stored in

In [3]:
!nvidia-smi

Wed Jul 21 12:33:34 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
import math
import torch
import torch.nn as nn
import torchvision
import numpy as np
import lightly
import os

In [7]:
num_workers = 2
batch_size = 64
seed = 1993
epochs = 50
input_size = 224

# dimension of the embeddings
num_ftrs = 512
# dimension of the output of the prediction and projection heads
out_dim = proj_hidden_dim = 512
# the prediction head uses a bottleneck architecture
#pred_hidden_dim = 128
# use 2 layers in the projection head
num_mlp_layers = 2

In [6]:
torch.manual_seed(0)
np.random.seed(0)

input_dir = '/content/drive/MyDrive/Colab Notebooks/atmacup/atmacup11/data/inputs/'
path_to_data = os.path.join(input_dir, 'photos')
model_dir = '/content/drive/MyDrive/Colab Notebooks/atmacup/atmacup11/data/model/'

## DataLoader

In [8]:
# define the augmentations for self-supervised learning
collate_fn = lightly.data.ImageCollateFunction(
    input_size=input_size,
    # require invariance to flips and rotations
    hf_prob=0.5,
    vf_prob=0.5,
    rr_prob=0.5,
    # satellite images are all taken from the same height
    # so we use only slight random cropping
    min_scale=0.5,
    # use a weak color jitter for invariance w.r.t small color changes
    cj_prob=0.2,
    cj_bright=0.1,
    cj_contrast=0.1,
    cj_hue=0.1,
    cj_sat=0.1,
)

# create a lightly dataset for training, since the augmentations are handled
# by the collate function, there is no need to apply additional ones here
dataset_train_simsiam = lightly.data.LightlyDataset(
    input_dir=path_to_data
)

# create a dataloader for training
dataloader_train_simsiam = torch.utils.data.DataLoader(
    dataset_train_simsiam,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_fn,
    drop_last=True,
    num_workers=num_workers
)

# create a torchvision transformation for embedding the dataset after training
# here, we resize the images to match the input size during training and apply
# a normalization of the color channel based on statistics from imagenet
test_transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((input_size, input_size)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(
        mean=lightly.data.collate.imagenet_normalize['mean'],
        std=lightly.data.collate.imagenet_normalize['std'],
    )
])



# create a lightly dataset for embedding
dataset_test = lightly.data.LightlyDataset(
    input_dir=path_to_data,
    transform=test_transforms
)



# create a dataloader for embedding
dataloader_test = torch.utils.data.DataLoader(
    dataset_test,
    batch_size=batch_size,
    shuffle=False,
    drop_last=False,
    num_workers=num_workers
)

In [9]:
!pip install efficientnet_pytorch

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=acefff47851058cfcee5f603185ebc59d64ba8ebf44f5dd3c6f462f5c9fb69f2
  Stored in directory: /root/.cache/pip/wheels/0e/cc/b2/49e74588263573ff778da58cc99b9c6349b496636a7e165be6
Successfully built efficientnet-pytorch


## Model

In [10]:
# we use a pretrained resnet for this tutorial to speed
# up training time but you can also train one from scratch
# Do not use pretrained Model
# resnet = torchvision.models.resnet18(pretrained=False)
# backbone = nn.Sequential(*list(resnet.children())[:-1])
from efficientnet_pytorch import EfficientNet
effB3 = EfficientNet.from_name('efficientnet-b3')
backbone = nn.Sequential(*list(effB3.children())[:-3])

# create the SimSiam model using the backbone from above
model = lightly.models.SimSiam(
    backbone,
    num_ftrs=num_ftrs,
#     proj_hidden_dim=pred_hidden_dim,
#     pred_hidden_dim=pred_hidden_dim,
#     out_dim=out_dim,
    num_mlp_layers=num_mlp_layers
)

In [11]:
#effB3

In [12]:
#backbone

In [13]:
#nn.Sequential(*list(effB3.children())[:-3])

In [14]:
#resnet = torchvision.models.resnet18(pretrained=False)
#backbone = nn.Sequential(*list(resnet.children())[:-1])
#backbone

In [15]:
#resnet

## Loss / Optimizer

In [16]:
# SimSiam uses a symmetric negative cosine similarity loss
criterion = lightly.loss.SymNegCosineSimilarityLoss()

# scale the learning rate
lr = 0.05 * batch_size / 256
# use SGD with momentum and weight decay
optimizer = torch.optim.SGD(
    model.parameters(),
    lr=lr,
    momentum=0.9,
    weight_decay=5e-4
)

In [17]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

avg_loss = 0.
avg_output_std = 0.
for e in range(epochs):

    for (x0, x1), _, _ in dataloader_train_simsiam:

        # move images to the gpu
        x0 = x0.to(device)
        x1 = x1.to(device)

        # run the model on both transforms of the images
        # the output of the simsiam model is a y containing the predictions
        # and projections for each input x
        y0, y1 = model(x0, x1)

        # backpropagation
        loss = criterion(y0, y1)
        loss.backward()

        optimizer.step()
        optimizer.zero_grad()

        # calculate the per-dimension standard deviation of the outputs
        # we can use this later to check whether the embeddings are collapsing
        output, _ = y0
        output = output.detach()
        output = torch.nn.functional.normalize(output, dim=1)

        output_std = torch.std(output, 0)
        output_std = output_std.mean()

        # use moving averages to track the loss and standard deviation
        w = 0.9
        avg_loss = w * avg_loss + (1 - w) * loss.item()
        avg_output_std = w * avg_output_std + (1 - w) * output_std.item()

    # the level of collapse is large if the standard deviation of the l2
    # normalized output is much smaller than 1 / sqrt(dim)
    collapse_level = max(0., 1 - math.sqrt(out_dim) * avg_output_std)
    # print intermediate results
    print(f'[Epoch {e:3d}] '
        f'Loss = {avg_loss:.2f} | '
        f'Collapse Level: {collapse_level:.2f} / 1.00')

NotImplementedError: ignored

In [ ]:
from torchsummary import summary
summary(model, (3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

In [ ]:
torch.save(model.state_dict(), os.path.join(model_dir,'simsiam_effB3.pth'))

In [ ]:
test = model.load_state_dict(torch.load(os.path.join(model_dir,'simsiam_effB3.pth')))

In [ ]:
from torchsummary import summary
#model = resnet18(pretrained=False)
resnet = resnet18(pretrained=False)
backbone = nn.Sequential(*list(resnet.children())[:-1])
model = lightly.models.SimSiam(
    backbone,
    num_ftrs=num_ftrs,
#     proj_hidden_dim=pred_hidden_dim,
#     pred_hidden_dim=pred_hidden_dim,
#     out_dim=out_dim,
    num_mlp_layers=num_mlp_layers
)
model.load_state_dict(torch.load(os.path.join(model_dir,'simsiam_res18_256.pth')))
model = model.backbone
model.add_module('flatten', nn.Flatten())
model.add_module('fc', nn.Linear(in_features=512, out_features=1, bias=True))
DEVICE = torch.device("cuda")
model.to(DEVICE)
summary(model, (3,224,224))

AttributeError: ignored